In [1]:
#import dependencies
import pandas as pd
import numpy as np
import datetime as dt

from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
from pprint import pprint

#import US data for table
CA_youtube_data = "CAvideos.csv"
CA_youtube = pd.read_csv(CA_youtube_data)

CA_youtube.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [2]:
CA_ytclean = CA_youtube.drop(columns=['thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description'])
CA_ytclean.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067


In [3]:
#import CA category json file
CA_json = "CA_category_id.json"
CA_category = pd.read_json(CA_json)
CA_category.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
1,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
2,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
3,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
4,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."


In [4]:
#appending category id to dict
CA_cats = []

for x in CA_category['items']:
        CA_cats.append([x['id'],x['snippet']['title']])
            
CA_cats

[['1', 'Film & Animation'],
 ['2', 'Autos & Vehicles'],
 ['10', 'Music'],
 ['15', 'Pets & Animals'],
 ['17', 'Sports'],
 ['18', 'Short Movies'],
 ['19', 'Travel & Events'],
 ['20', 'Gaming'],
 ['21', 'Videoblogging'],
 ['22', 'People & Blogs'],
 ['23', 'Comedy'],
 ['24', 'Entertainment'],
 ['25', 'News & Politics'],
 ['26', 'Howto & Style'],
 ['27', 'Education'],
 ['28', 'Science & Technology'],
 ['30', 'Movies'],
 ['31', 'Anime/Animation'],
 ['32', 'Action/Adventure'],
 ['33', 'Classics'],
 ['34', 'Comedy'],
 ['35', 'Documentary'],
 ['36', 'Drama'],
 ['37', 'Family'],
 ['38', 'Foreign'],
 ['39', 'Horror'],
 ['40', 'Sci-Fi/Fantasy'],
 ['41', 'Thriller'],
 ['42', 'Shorts'],
 ['43', 'Shows'],
 ['44', 'Trailers']]

In [5]:
CA_categories = pd.DataFrame.from_dict(CA_cats)
CA_categories.columns = ['category_id', 'category']
CA_categories

,category_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [6]:
CA_ytclean.convert_dtypes()
CA_ytclean.dtypes

video_id         object
trending_date    object
title            object
channel_title    object
category_id       int64
publish_time     object
tags             object
views             int64
likes             int64
dislikes          int64
comment_count     int64
dtype: object

In [7]:
CA_categories.convert_dtypes()
CA_categories.dtypes

category_id    object
category       object
dtype: object

In [8]:
CA_categories['category_id'] = pd.to_numeric(CA_categories['category_id'])
CA_categories.dtypes

category_id     int64
category       object
dtype: object

In [9]:
CA_final = pd.merge(CA_ytclean, CA_categories, on="category_id")
CA_final.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,category
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,Music
1,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,Music
2,H1KBHFXm2Bg,17.14.11,21 Savage - Bank Account (Official Music Video),21 Savage,10,2017-11-10T19:00:02.000Z,"21 savage|""bank account""|""21 savage bank accou...",5068229,263596,8585,28976,Music
3,7MxiQ4v0EnE,17.14.11,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,10,2017-11-11T16:41:15.000Z,"punjabi songs|""punjabi bhangra""|""punjabi music...",5718766,127477,7134,8063,Music
4,j67FgNEvyh8,17.14.11,Telefoon || Babbu Maan || Promo || Full Song 1...,Hey Yolo,10,2017-11-12T04:24:34.000Z,"Telefoon|""Babbu Maan""|""Mehfil Mitran Di""|""new ...",178447,4339,96,876,Music


In [11]:
CA_final['Country'] = 'CA'
CA_final

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,category,Country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,Music,CA
1,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,Music,CA
2,H1KBHFXm2Bg,17.14.11,21 Savage - Bank Account (Official Music Video),21 Savage,10,2017-11-10T19:00:02.000Z,"21 savage|""bank account""|""21 savage bank accou...",5068229,263596,8585,28976,Music,CA
3,7MxiQ4v0EnE,17.14.11,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,10,2017-11-11T16:41:15.000Z,"punjabi songs|""punjabi bhangra""|""punjabi music...",5718766,127477,7134,8063,Music,CA
4,j67FgNEvyh8,17.14.11,Telefoon || Babbu Maan || Promo || Full Song 1...,Hey Yolo,10,2017-11-12T04:24:34.000Z,"Telefoon|""Babbu Maan""|""Mehfil Mitran Di""|""new ...",178447,4339,96,876,Music,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40802,l7E0kTvARsA,18.01.06,Golak Bugni Bank Te Batua Full Movie (HD) | Ha...,Rhythm Boyz,30,2018-05-31T13:30:04.000Z,"full movie|""full punjabi movie""|""golak""|""bugni...",781977,22027,783,1096,Movies,CA
40803,l7E0kTvARsA,18.02.06,Golak Bugni Bank Te Batua Full Movie (HD) | Ha...,Rhythm Boyz,30,2018-05-31T13:30:04.000Z,"full movie|""full punjabi movie""|""golak""|""bugni...",2289027,41807,1811,2153,Movies,CA
40804,l7E0kTvARsA,18.03.06,Golak Bugni Bank Te Batua Full Movie (HD) | Ha...,Rhythm Boyz,30,2018-05-31T13:30:04.000Z,"full movie|""full punjabi movie""|""golak""|""bugni...",3523444,52355,2520,3000,Movies,CA
40805,l7E0kTvARsA,18.04.06,Golak Bugni Bank Te Batua Full Movie (HD) | Ha...,Rhythm Boyz,30,2018-05-31T13:30:04.000Z,"full movie|""full punjabi movie""|""golak""|""bugni...",4638549,60148,3160,3441,Movies,CA
